In [1]:
import ujson
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import collections
import pickle
from konlpy.tag import Komoran


komoran = Komoran()

def read_documents(input_file_name):
    """주어진 이름의 파일에서 소요리 문답 질문을 읽어서 List의 List객체로 돌려준다."""
    
    qna_documents = []

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            json_obj = ujson.loads(line)

            qna_document = json_obj["question"]
            qna_documents.append(qna_document)
            
    return qna_documents

def load_catechism_vectorizer(catechism_vectorizer_pkl_filename):
    """소요리문답 자료로부터 문서-단어 행렬을 생성하여 저장한 벡터자료를 적재"""
    vectorizer = TfidfVectorizer(tokenizer=str.split)
    
    #load the doc_term_mat
    with open(catechism_vectorizer_pkl_filename, 'rb') as handle:
        vectorizer = pickle.load(handle)

    words = vectorizer.get_feature_names()
    
    return vectorizer, words


def load_catechism_kmeans_model(kmeans_pkl_filename):
    """학습된 K-Means모델로부터 예측을 수행한다."""
    print("\n")
    print("Loading the saved catechism KMeans clustering model")
    
    # Loading the saved KMeans clustering model pickle
    kmeans_model_pkl = open(kmeans_pkl_filename, 'rb')
    kmeans_model_model = pickle.load(kmeans_model_pkl)
    n_clusters = kmeans_model_model.get_params()['n_clusters']
    print("Loaded kmeans model :: {}".format(kmeans_model_model))
    print("n_clusters :: {}".format(kmeans_model_model.get_params()['n_clusters']))

    return kmeans_model_model, n_clusters

def print_centroid_words(model, n_clusters, words, qna_documents):
    """주어진 모델과 단어들을 이용하여 군집별 중심 단어를 출력한다."""
    
    print("\n")
    print("군집별 중심 단어")
    print()

    ordered_centroids = model.cluster_centers_.argsort()[:, ::-1]
    #print("ordered_centroids=[{}]".format(ordered_centroids))
    clusters = collections.defaultdict(list)
    for i, label in enumerate(model.labels_):
        #print("문장번호=[{}],클러스터번호=[{}]".format(i,label))
        clusters[label].append(i)
    


    for cluster_num in range(n_clusters):
        center_word_nums = [word_num for word_num 
                            in ordered_centroids[cluster_num, :20]]
        #print("center_word_nums=[{}]".format(center_word_nums))
        center_words = [words[word_num] for word_num in center_word_nums]
        print("군집 키워드 {}: {}".format(cluster_num, ", ".join(center_words)))
        print()
        for i,sentence in enumerate(clusters[cluster_num]):
            print("\t문장  {} :문답번호=[{}]{}".format(i, sentence+1, qna_documents[sentence]))
        print("="*100)

def get_prediction_by_kmeans_model(sentence, vectorizer, kmeans_model_model):
    """학습된 K-Means모델로부터 예측을 수행한다."""
    print("\n")
    print("Prediction")
    
    Y = vectorizer.transform(sentence)
    prediction = kmeans_model_model.predict(Y)
    print(prediction)

def get_query_sentence(analyzer, text):
    query_sentence = analyzer.morphs(text)
    return_sentence = []
    work_sentence = ""
    for word in query_sentence:
        work_sentence += word + " "
        
    return_sentence.append(work_sentence.strip())
    print(return_sentence)
    return return_sentence

    
def main():
    """K-Means모델로 군집화한 소요리문답 자료를 이용 주어진 문장의 군집 그룹을 결정한다."""
    
    input_file_name = "data/catechism/catechism.ma.txt"

    # Dump the trained KMeans clustering with Pickle
    catechism_vectorizer_pkl_filename = 'catechism_vectorizer.pkl'
    kmeans_pkl_filename = 'kmeans_pkle_clustering_20171129.pkl'

    text = input("분류할 문장을 입력하세요: ")
    
    #sentence = ["사람 의 제일 되는 목적 이 무엇 이 ㄴ가 ?"]
    sentence = []
    sentence = get_query_sentence(komoran, text)
    print("sentence[{}]".format(sentence))


    qna_documents = read_documents(input_file_name)
    vectorizer, words = load_catechism_vectorizer(catechism_vectorizer_pkl_filename)
    model, n_clusters = load_catechism_kmeans_model(kmeans_pkl_filename)
    print_centroid_words(model, n_clusters, words, qna_documents)
    get_prediction_by_kmeans_model(sentence, vectorizer, model)
        

# 실행
#main()

In [2]:
# 실행
main()

분류할 문장을 입력하세요: 사람의 제일되는 목적
['사람 의 제일 되 는 목적']
sentence[['사람 의 제일 되 는 목적']]


Loading the saved catechism KMeans clustering model
Loaded kmeans model :: KMeans(n_clusters=11, verbose=1)
n_clusters :: 11


군집별 중심 단어

군집 키워드 0: 성찬, 신약, 주의, 참여, 합당, 세례, 주님, 즙, 포도, 떡, 죽, 정하, 나타내, 무엇, 곧, 받, 마시, 스스로, 살피, 자청

	문장  0 :문답번호=[93]신약의 성례가 무엇인가?
	문장  1 :문답번호=[96]주의 성찬이 무엇인가?
	문장  2 :문답번호=[97]주의 성찬에 합당하게 참여하려면 어떻게 하여야 하는가?
군집 키워드 1: 하나님, 사람, 계명, 제일, 본분, 요구, 행동, 치, 되, 권능, 계시, 무엇, 아들, 지키, 목적, 지혜, 보존, 영원, 신격, 삼위

	문장  0 :문답번호=[1]사람의 제일 되는 목적이 무엇인가?
	문장  1 :문답번호=[3]성경이 제일 요긴하게 교훈하는 것이 무엇인가?
	문장  2 :문답번호=[4]하나님은 어떤 분이신가?
	문장  3 :문답번호=[5]하나님 한분밖에 또 다른 하나님이 계신가?
	문장  4 :문답번호=[6]하나님의 신격에 몇 위가 계신가?
	문장  5 :문답번호=[11]하나님의 섭리하시는 일이 무엇인가?
	문장  6 :문답번호=[21]하나님의 선택하신 자의 구속자가 누구신가?
	문장  7 :문답번호=[22]그리스도께서 하나님의 아들로서 어떻게 사람이 되셨는가?
	문장  8 :문답번호=[39]하나님께서 사람에게 요구하시는 본분이 무엇인가?
	문장  9 :문답번호=[50]제 이 계명이 명하는 것이 무엇인가?
	문장  10 :문답번호=[53]제 삼 계명이 무엇인가?
	문장  11 :문답번호=[64]제 오 계명이 명하는 것은 무엇인가?
	문장  12 :문답번호=[71]제 칠 계명이 명하는 것은 무엇인가?